In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [63]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [64]:
df = pd.read_csv('../datasets/Airline_Passenger_Satisfaction.csv')

In [65]:
df.shape

(103904, 25)

In [66]:
bdf = BaseDatasetTransform(df,target='satisfaction')

In [67]:
df,categorical_features = bdf.fit_transform()


Удаление колонок с именем "id": ['id']
-------------------------------------------

В колонке satisfaction нет пропущенных значений
-------------------------------------------
В наборе данных есть пропущенные значения:
Колонки с пропущенными значениями:
Arrival Delay in Minutes    310
dtype: int64
Все пропущенные значения заполнены.
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Gender', 'Customer Type', 'Type of Travel', 'Class'], dtype='object')

Числовые колонки:
Index(['Unnamed: 0', 'Age', 'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes',
       'Arrival Delay in Minutes'],
      dtype='object

In [68]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям
satisfaction
neutral or dissatisfied    58879
satisfied                  45025
Name: count, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n
Разделение датасета выполнено успешно


In [69]:
X_train

,Unnamed: 0,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
2155,2155,Female,Loyal Customer,59,Business travel,Eco,1931,2,2,2,...,3,1,1,2,1,2,1,1,64,35.0
89944,89944,Female,Loyal Customer,52,Business travel,Eco Plus,442,1,5,5,...,3,1,1,1,1,3,1,2,4,0.0
61097,61097,Female,disloyal Customer,20,Business travel,Business,925,4,5,4,...,4,4,5,5,4,3,4,4,17,6.0
76313,76313,Female,Loyal Customer,27,Personal Travel,Eco,438,3,2,3,...,3,3,4,3,2,4,2,3,2,0.0
92617,92617,Male,Loyal Customer,35,Business travel,Eco,776,4,3,3,...,4,4,4,4,1,2,1,4,5,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63485,63485,Male,disloyal Customer,36,Business travel,Business,368,2,2,2,...,3,3,3,5,5,5,5,3,26,25.0
90214,90214,Female,Loyal Customer,46,Business travel,Business,1884,5,5,5,...,4,4,4,4,4,5,4,5,48,41.0
69790,69790,Male,Loyal Customer,23,Business travel,Business,1685,4,4,4,...,4,4,4,3,5,5,4,4,0,0.0
32633,32633,Male,disloyal Customer,33,Business travel,Eco,1585,2,2,2,...,5,1,4,4,3,3,3,1,26,30.0


In [70]:
lgbm_model = lgb.LGBMClassifier(verbose=-1)

columns_to_delete = ['Unnamed: 0']  


model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='binary',
    metric='binary_error'
)

In [71]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['Gender',
                                                                  'Customer '
                                                                  'Type',
                                                                  'Type of '
                                                                  'Travel',
                                                                  'Class'],
                                            columns_to_delete=['Unnamed: 0'],
                                            model=LGBMClassifier(metric='binary_error',
                                                                 objective='binary',
                                                                 verbose=-1)))

In [72]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.962434
1,precision,0.963852
2,recall,0.959773
3,f1,0.961594
